<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula14a_hugging_face_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  6631k      0  0:00:12  0:00:12 --:--:-- 15.5M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [2]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [3]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="textattack/bert-base-uncased-imdb")

In [6]:
pipe("This is a great movie!")

[{'label': 'LABEL_1', 'score': 0.9665761590003967}]

In [7]:
from sklearn.metrics import accuracy_score
import numpy as np
import time

In [8]:
ypred = []
test_labels = []
start = time.time()
for batch, (inputs, targets) in enumerate(test_ds):
  for i in range(len(inputs)):
    yp = pipe(inputs[i].numpy().decode('UTF-8')[:512])
    test_labels.append(int(targets[i].numpy()==1))
    ypred.append(int(yp[0]['label']=="LABEL_1"))
  print(batch, time.time()-start, accuracy_score(test_labels, ypred))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0 0.5172951221466064 0.9375
1 1.135636329650879 0.90625
2 2.355058431625366 0.90625
3 3.574014902114868 0.875
4 4.451106548309326 0.86875
5 5.588292360305786 0.8541666666666666
6 6.6646013259887695 0.8660714285714286
7 7.392066478729248 0.8671875
8 8.04172158241272 0.8680555555555556
9 8.396782875061035 0.871875
10 8.748668670654297 0.8664772727272727
11 9.122866153717041 0.8723958333333334
12 9.464287281036377 0.8701923076923077
13 9.823973178863525 0.875
14 10.199789762496948 0.875
15 10.549949407577515 0.875
16 10.909099102020264 0.8786764705882353
17 11.286092519760132 0.8836805555555556
18 11.645628929138184 0.8865131578947368
19 12.01257586479187 0.8875
20 12.3696768283844 0.8898809523809523
21 12.713402271270752 0.8892045454545454
22 13.085731744766235 0.8858695652173914
23 13.438133001327515 0.88671875
24 13.943358898162842 0.8875
25 14.42514681816101 0.8870192307692307
26 14.89076042175293 0.8912037037037037
27 15.456195831298828 0.890625
28 15.955626726150513 0.88793103448275